# Imputing missing values

We work with the data set how it is present now and apply two disparate methods, each computing the imputations for missing values differently. 

1) The first method is a **weighted k nearest neighbour (w-kNN)** algorithm, which imputes missing values with weights equal to the inverse Euclidean distance. 

**Assumptions:** we believe the values missing lie in between the boundaries of the highest and lowest value present in the data set. This might work well for most, but it could also be that the missing values are outliers.

2) The second method is **Gaussian process (GP) regression** where we take the mean function as the imputation for missing values.

**Assumptions:** we believe underlying time-series are continuous, i.e. we can have smooth approximations of the available time-series.

### `groupings` and `countries`

We want to **remind** ourselves that our list `groupings` does not only contain countries, it also contains groups of countries like continents, economic zones, etc. and islands which belong to certain countries, but could somehow be very different as e.g. territories. We save all these entries in a new list `countries`. 

Furthermore, some sub-indicators are of ordinal type, i.e. they are defined as 'number of countries which...'. For all countries, we have here either a 1 or a 0 for 'yes' or 'no', respectively. For all other groupings, we have there most likely a number larger than 1. This could be tricky for our imputations later, because these are based on the similarity of two groupings and these similarities could be strong between, say, France and the World Trade Organisation (WTO). If the WTO had a missing value for an ordinal sub-indicator, the imputation would most likely be very similar to the one of France, so 1 or 0. But this is unrealistic, and all other countries being member of the WTO and behave almost as similar as France, would make the imputation just closer to 1 and not larger than 1.

The solution is simple, but time-consuming: we only impute missing values for `countries`. Then, we compute the averages and sums for continuous and ordinal sub-indicators, respectively, of the countries which form these non-country groupings. 

It's time-consuming because we need to do it by hand. I prefer doing it in a GUI and export the csv file to be able to quickly delete non-countries from the list `groupings`.

In [ ]:
import numpy as np
import pandas as pd
import math
import os
import pickle
import copy
import itertools

In [ ]:
# CHECKPOINT (we don't want to re-run the entire script every time we continue working on it)
dict_all_s = pickle.load(open('dict_all_s.pkl', 'rb'))

In [ ]:
# check
print('Standardised values: ')
print(dict_all_s['Africa']['2000'])

Now, we open the `csv` file in a GUI and delete the groupings which are *not* countries or part of countries. We call these non-country groupings and examples are North America, Western Asia, Least Developed Countries (LDC), Land Locked Developing Countries (LLDC), Small Island Developing States (SIDS).

In [ ]:
# read amended csv file
c = pd.read_csv('utils/countries.csv', dtype=str)
countries = list(c['Countries'])

#check
countries

## 1) weighted k nearest neighbour (w-kNN)

The w-kNN algorithm is straightforward: we calculate how similar countries are in each given year $y$ with the standardised Euclidean distance $E_y$ and take the inverse of the absolute $|E_y|$ as the weight to impute missing values in any given country $c_i$ in each given year $y$ for each given sub-indicator $j$.

A good start to understand this algorithm is to understand high-dimensional space: https://youtu.be/wvsE8jm1GzE

### Euclidean distance
The Euclidean distance $e_y$ for year $y$ for any given pair of countries $(c_i, c_k)$ for any given sub-indicator $j$ is calculated by:
$$ e_y(c_{i}, c_{k}) = \lVert c_{i}, c_{k} \rVert_2 = \sqrt{(c_{ij} - c_{kj})^2} $$

We calculate the squared distances between any given pair of countries $(c_i, c_k)$, but do not consider the country $k+1$ which has the largest distance $e_y$ to country $i$. We do so for any given sub-indicator $j$ and take the square root of it. $c_{ij}$ is the sub-indicator $j$ of country $i$, and $i \neq k$. Thus, any unique pair of countries $i$ an and $k$, $i \neq k$, has **one** Euclidean distance $e_y$ for year $y$ only.

Afterwards, we standardise this with respect to the country $k+1$ which has the largest distance $e_y$ to country $i$ by the following equation:
$$ E_y(c_{i}, c_{k}) = \frac{e_y(c_{i}, c_{k})}{e_y(c_{i}, c_{k+1})} $$


### Imputations
We want that our imputations $x^{j'}_{i,y}$ for missing sub-indicator $j'$ in country $i$ in year $y$ are similar to sub-indicators $j$ of countries $k$ which have a **small** Euclidean distance $E_y$ and dissimilar to sub-indicators $j$ of countries $k$ which have a **large** Euclidean distance $E_y$. Consequently, the imputations $x^{j'}_{i,y}$ are the *weighted* averages where the *weights* are equal to the inverse standardised Euclidean distance $\frac{1}{|E_y(c_{i}, c_{k})|}$.

First, we compute $E_y$ for all available pairs of sub-indicators $j$ amongst two countries $i$ and $k$. Since countries have different amounts of available data points, we average by multiplying the sum by $1/J$, where $J$ is the total number of sub-indicators taken into account here. Note, this does not necessarily be 375, because we have missing values for many sub-indicators. Second, we sum over $k$ to add together all weighted $x^j_{k,y}$ of each unique pair of countries $i$ and $k$ and compute its average by dividing by $K$.

$$ x^{j'}_{i,y} = \frac{1}{K} \sum_k \frac{1}{|E_y(c_{i}, c_{k})|} \cdot x^j_{k,y} $$

**Assumptions:** we calculate how similar countries are according to their values for *all* sub-indicators in a given year. We assume that the specific sub-indicators which do not have values in this given year are exactly as similar as the ones which we can calculate a distance for.

Let's have a final check before we start our w-kNN algorithm to compute all missing values. We have here negative values which might sound confusing, but bear in mind that we have standardised the data before, i.e. the data distribution has mean 0 and standard deviation 1.

In [ ]:
dict_all_s['Iraq (Central Iraq)'].head()

In [ ]:
# CHECKPOINT
dict_e = pickle.load(open('utils/distances_unstand.pkl', 'rb'))
dict_E = pickle.load(open('utils/distances_stand.pkl', 'rb'))

In [ ]:
#check
print('Euclidean distance: ', dict_E[('Azerbaijan', 'Angola', '2000')])

### Calculating the Euclidean distance

We can calculate the standardised distance $E_y(c_{i}, c_{k})$ after having prepared everything. We do this for each unique pair of two *countries* in each year. In other words, we do not want to calculate $E_y(c_{i}, c_{k})$ for $i = k$ and $E_y(c_{i}, c_{k}) = E_y(c_{k}, c_{i})$.

The python package <code>itertools</code> can help us generating the unique pairs of countries.

In [ ]:
# create list out of all unique combinations
countrycombinations = list(itertools.combinations_with_replacement(countries, 2))
countrycombinations

In [ ]:
# check
countrycombinations[0][0]

Here, we calculate the standardised distance $E_y(c_{i}, c_{k})$ for each unique pair of two countries in each year.

In [ ]:
from scipy.spatial.distance import euclidean

First, we compute the (not standardised) distances $e_y$ and insert them into a new dictionary `dict_e`.

While exploring the data, we see that nearly no data are available for the years `1990` to `1999`. Consequently, imputations in those years will be based on very weak foundations and we do not consider these years for now. For our similarity investigations later, it does not matter much how many data points we have totally available per country, it is more important that all countries have the same amount of data points. For now, we also omit data for the year `2019`, because it seems not all countries have reported their data yet. Hence, there aren't too many data points available neither.

We set the `period` of years we want to consider in our computations for $e_y$.

In [ ]:
period = ['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']

In [ ]:
# call seriescodes again
info = pd.read_csv('utils/info.csv', dtype=str)
seriescodes = list(info['SeriesCode'])
seriescodes

In [ ]:
# very memory-intensive (~3 hours computing time)
# no need to run every time again, just see CHECKPOINT above and load pickle file

dict_e = {}    

for year in period:
    
    for countrycombination in countrycombinations:
        
        country0_e = []    # create two empty lists for the two groupings we consider at the moment
        country1_e = []    # these lists contain series codes with data available in both groupings
        j = 0    # counter
        
        for seriescode in seriescodes:
            # we can only consider sub-indicators with data available in both groupings
            if pd.isna(dict_all_s[countrycombination[0]][year][seriescode]) is False and pd.isna(dict_all_s[countrycombination[1]][year][seriescode]) is False:
                country0_e.append(dict_all_s[countrycombination[0]][year][seriescode])
                country1_e.append(dict_all_s[countrycombination[1]][year][seriescode])
                
                j += 1
        
        print('number of data points available: ', j)    # check
        if j > 0:
            e = euclidean(country0_e, country1_e)
        else:
            e = np.nan    # make NaN, so we can later assign 1/E = 0 easily
            j = 1
            
        print('e in {} between {} and {}:'.format(year, countrycombination[0], countrycombination[1]), e/j)
        
        dict_e[year, countrycombination[1], countrycombination[0]] = e/j
        dict_e[year, countrycombination[0], countrycombination[1]] = dict_e[year, countrycombination[1], countrycombination[0]]


In [ ]:
# better save this precious data
f = open('utils/distances_unstand.pkl', 'wb')
pickle.dump(dict_E, f)
f.close()

In [ ]:
# check
dict_e['2000', 'British Virgin Islands', 'Martinique']

Standardise these distances $e_y$ and save them in `dict_E`:

In [ ]:
dict_E = copy.deepcopy(dict_e)

for year in period:
    
    list_e = []    # list with all distances in each year
    max_e = 0    # maximum value in list_e
    dict_e_year = {}    # auxiliary dictionary per year
    
    
    for k in dict_e.keys():
        if year in k:
            dict_e_year[k] = dict_e[k]
            

    max_e = np.nanmax(list(dict_e_year.values()))
    print('------------')
    print('max_e in {}'.format(year), max_e)
    print('------------')
    list(dict_e_year.values()).remove(max_e)
    
    
    for k in dict_e_year.keys():
        print(k)
        if np.isnan(dict_e_year[k]) == False:
            print('unstandardised distance e:', dict_e_year[k])
            dict_E[k] = dict_e_year[k] / max_e    # standardise distance
            
        else:
            dict_E[k] = 999999999999
        
        print('standardised distance E:', dict_E[k])
        print('------------')


In [ ]:
# check
print('unstandardised distance e:', dict_e['2000', 'Afghanistan', 'Colombia'])
print('standardised distance E:  ', dict_E['2000', 'Afghanistan', 'Colombia'])

In [ ]:
# better save this precious data
f = open('utils/distances_stand.pkl', 'wb')
pickle.dump(dict_E, f)
f.close()

### Imputations for countries

Now, we impute the missing values according to the equation we previously derived:

$$ x^{j'}_{i,y} = \frac{1}{K} \sum_k \frac{1}{|E_y(c_{i}, c_{k})|} \cdot x^j_{k,y} $$

To recap, our imputations $x^{j'}_{i,y}$ for missing sub-indicator $j'$ in country $i$ in year $y$ should be similar to sub-indicators $j$ of country $k$, according to the inverse standardised Euclidean distance $\frac{1}{|E_y(c_{i}, c_{k})|}$ between $i$ and $k$. 

As aforementioned and shown in the equation of $E_y$, $E_y$ is dependent on the number of pairs we have in both countries data for. Our `dict_E` has already entries for $E$ normalised according to this number of available pairs of data. We multiply our weight for each imputation, i.e. the inverse standardised Euclidean distance $\frac{1}{|E_y(c_{i}, c_{k})|}$ between $i$ and $k$, by the value $x^j_{k,y}$ of the other country $k$ in year $y$ for sub-indicator $j$. We sum over $k$ to add together all weighted $x^j_{k,y}$ of each unique pair of countries $i$ and $k$ and compute its average by dividing by $K$. $K$ is the number of countries which have values available for the sub-indicator $x^{j'}_{k,y}$ to be computed. 

We also know that some indicators are binary, i.e. 1 for 'yes' and 0 for 'no', and ordinal for groupings of countries. These indicators start usually with 'number of countries which...'. The imputations for these must be handled differently: we round the imputed value to an integer.

In [ ]:
# CHECKPOINT
dict_all_i = pickle.load(open('utils/dict_all_i.pkl', 'rb'))

In [ ]:
ordinal = ['1.5.3','12.1.1','15.6.1','17.16.1','17.18.2','17.18.3','17.19.2']

In [ ]:
# series codes for ordinal indicators
ordinal_ser = ['SG_DSR_LEGREG','SG_DSR_LGRGSR','SG_SCP_CNTRY','SG_SCP_CORMEC','SG_SCP_MACPOL','SG_SCP_POLINS','ER_CBD_ABSCLRHS','ER_CBD_NAGOYA','ER_CBD_ORSPGRFA','ER_CBD_PTYPGRFA','ER_CBD_SMTA','SG_PLN_MSTKSDG','SG_STT_FPOS','SG_STT_NSDSFDDNR','SG_STT_NSDSFDGVT','SG_STT_NSDSFDOTHR','SG_STT_NSDSFND','SG_STT_NSDSIMPL','SG_REG_BRTH90','SG_REG_BRTH90N','SG_REG_CENSUS','SG_REG_CENSUSN','SG_REG_DETH75','SG_REG_DETH75N']

In [ ]:
# memory-intensive (~3 hours)

dict_all_i = copy.deepcopy(dict_all_s)

for country in countries: 
    
    not_countries = [c for c in countries if c != country]
    
    for seriescode in seriescodes:
        for year in period:            
            if pd.isna(dict_all_s[country][year][seriescode]) is True:
                K = 0
                all_k = []
                
                for not_country in not_countries: 
                    if pd.isna(dict_all_s[not_country][year][seriescode]) is False:    # not_country can also be NaN -> exclude those
                        K += 1
                        k = 1/(dict_E[(year, country, not_country)]) * dict_all_s[not_country][year][seriescode]
                        all_k.append(k)
                        sum_k = np.sum(all_k)
                    
                print('K =', K)
                    
                if K > 0:
                    print('sum k =', sum_k)
                                                                
                    if seriescode in ordinal_ser:
                        dict_all_i[country][year][seriescode] = np.around(sum_k / K)    # round to have integer
                    else:
                        dict_all_i[country][year][seriescode] = sum_k / K
                            
                else:
                    dict_all_i[country][year][seriescode] = np.nan
                    

                
                print('Imputation for {} in {} in {}'.format(seriescode, country, year), dict_all_i[country][year][seriescode])
                

In [ ]:
#check
print('NaN here', dict_all_s['Iraq (Central Iraq)']['2009']['SI_POV_DAY1'])
print('Imputed value', dict_all_i['Iraq (Central Iraq)']['2009']['SI_POV_DAY1'])

### Imputations for non-country groupings

These were the imputations for all countries, but we would like to have imputations for non-country groupings, too. As aforementioned, we compute the averages and sums for continuous and ordinal sub-indicators, respectively, of the countries which form these non-country groupings. 

Unfortunately, we need to define these non-country groupings by hand.

Finally, we can compute the sums and averages and have our imputations complete.

In [ ]:
non_countries = [g for g in groupings if g != countries]

for non_country in non_countries:
    
    for seriescode in seriescodes:
        for year in period:
            if pd.isna(dict_all_s[non_country][year][seriescode]) is True:
                    
                if seriescode in ordinal_ser:
                    dict_all_i[non_country][year][seriescode] = np.sum()
                else:
                    dict_all_i[non_country][year][seriescode] = np.mean()

We want to save the imputations to have another checkpoint here.

In [ ]:
# as csv files
if not os.path.exists('csv_imputed_s'):
    os.mkdir('csv_imputed_s')

for group in groupings:
    dict_all_i[group].to_csv(r'csv_imputed_s/{}.csv'.format(group))
    
# as pkl files
imp = open('utils/dict_all_i.pkl', 'wb')
pickle.dump(dict_all_i, imp)
imp.close()

## 2) Gaussian process regression

This method takes less assumptions into account and the assumptions it takes are much more realistic. Basically, we only assume that the data we have form continuous time-series. Given our socio-economic and environmental data, this is highly probable since nothing can drastically change over night in this world.

Gaussian processes (GPs) are useful for probabilistic modeling of unknown functions. Each of our underlying time-series which have missing values is an unknown function. GPs generalise the multivariate Gaussian distribution by treating any finite subset of function values as a multivariate, i.e. joint, Gaussian distributions. Any GP is fully specified by a mean vector $\mathbf{m}$ and covariance matrix $\mathbf{K}$, whose elements are computed by a mean function $m = m(\mathbf{x})$ and a covariance function $\mathbf{K}_{ij} = k(x_i, x_j)$, respectively, where $i$ and $j$ are the years we have measurements available in each time-series. We treat these measurements as random variables in GPs.

Conditioning the probability distributions for our imputations on this joint distribution of function values $f$ yields the Gaussian predictive distribution of function values $f'$ at times of missing values. 

$$p(f' | f) = \mathcal{N}(\mu, \mathbf{\Sigma})$$
$$\mu = \mathbf{m}_{f'} + \mathbf{K}_{f'f} \mathbf{K}_{ff}^{-1}(\mathbf{f}-\mathbf{m}_f)$$
$$\mathbf{\Sigma} = \mathbf{K}_{f'f'} - \mathbf{K}_{f'f} \mathbf{K}_{ff}^{-1} \mathbf{K}_{ff'}$$


*Some parts from hereon are slightly adapadted from a notebook written by Wil Ward, adapted from notebooks by [Rich Wilkinson](https://rich-d-wilkinson.github.io/) and [Neil Lawrence](http://inverseprobability.com/).*

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import GPy

Let's see first of all one exemplary time-series, `dict_all_i['Algeria'][period].loc['SI_POV_DAY1']`, to get a basic idea what we could expect.

In [ ]:
# making integeres out of floats 
per = []
for p in period:
    per.append(int(p))
# per

In [ ]:
# years from 2000 to 2018
x = np.array(per).reshape((-1, 1))

# standardised values for these X years
y = np.array(dict_all_i['Algeria'][period].loc['SI_POV_DAY1']).reshape((-1, 1))

# Setup our figure environment
plt.figure(figsize=(19, 7))

# Plot observations
plt.plot(x, y, "kx", mew=2)

# Annotate plot
plt.xlabel("years", size=20), plt.ylabel("standardised values", size=20)
plt.xticks(np.arange(min(x), max(x)+1, 1.0))
plt.legend(labels=["measurements"]);

We will start with defining a covariance function, from hereon referred to as a **kernel**. The most commonly used kernel in machine learning is the Gaussian-form radial basis function (RBF) kernel. 

The definition of the (1-dimensional) RBF kernel has a Gaussian-form, defined as:

$$
    \kappa_\mathrm{rbf}(x_i,x_j) = \sigma^2\exp\left(-\frac{(x_i-x_j)^2}{2\mathscr{l}^2}\right)
$$

It has two parameters, described as the variance, $\sigma^2$ and the lengthscale $\mathscr{l}$.

We define our kernels using the input dimension as the first argument, in the simplest case `input_dim=1` for 1-dimensional regression, how it is in our case. 

In [ ]:
# Create a 1-D RBF kernel with default parameters
k = GPy.kern.RBF(1)
# Preview the kernel's parameters
k

### Visualising the kernel

We can visualise our kernel in a few different ways. One possibility is to plot the *shape* of the kernel at one specific location along the $x$-axis, say $0$, by plotting $k(x,0)$ over some sample space $x$ which, looking at the equation above, clearly has a Gaussian shape. This describes the **covariance between each sample location and $0$** and we can see that it decreases the further we move away from $0$.

Alternatively, we can construct a full covariance matrix, $\mathbf{K}_{xx} := k(x_i,x_j)$ with samples $x_i = x_j$. The resulting GP prior is a multivariate normal distribution over the space of samples $x$: $\mathcal{N}(\mathbf{0}, \mathbf{K}_{xx})$. It should be evident then that the elements of the matrix represent the covariance between respective points in $x_i$ and $x_j$, and that it is exactly $\sigma^2 = 1$ in the diagonal.

We can show this using `pyplot` to plot the vector $k(x_i,0)$ and the matrix $k(x_i,x_j)$ using `k.K(`$\cdot$ `,` $\cdot$`)`.

We do this first for a too-perfect-for-reality toy data set and define it as $X$.

In [ ]:
# Our toy data set and sample space: 250 samples in the interval [-4,4]
X = np.linspace(-4.,4.,250)[:, None] # we need [:, None] to reshape X into a column vector for use in GPy

# Set up the plotting environment
plt.figure(figsize=(18,5))

# ==== k(x,0)

plt.subplot(121) # left plot

# First, sample kernel at x_j = 0
k.lengthscale = 1
K = k.K(X, np.array([[0.]])) # k(x,0)

# Plot covariance vector
plt.plot(X,K)

# Annotate plot
plt.xlabel("$x_i$", size=20), plt.ylabel("$\kappa$", size=20)
plt.title("$\kappa_{rbf}(x,0)$", size=20)

# ==== k(x_i,x_j)

plt.subplot(122) # right plot

# The kernel takes two inputs, and outputs the covariance between each respective point in the two inputs
K = k.K(X,X)

# Plot the covariance of the sample space
plt.pcolor(X.T, X, K)

# Format and annotate plot
plt.gca().invert_yaxis(), plt.gca().axis("image")
plt.xlabel("$x_i$", size=20), plt.ylabel("$x_j$", size=20), plt.colorbar()
plt.title("$\kappa_{rbf}(x_i,x_j)$", size=20);

Let's do the same now for one of our real time-series.

In [ ]:
# time-series for sub-indicator 'SI_POV_DAY1' in Algeria
X = np.array(dict_all_i['Algeria'][period].loc['SI_POV_DAY1']).reshape((-1, 1))

# Set up the plotting environment
plt.figure(figsize=(18,5))

# ==== k(x,0)

plt.subplot(121) # left plot

# First, sample kernel at x_j = 0
k.lengthscale = 1
K = k.K(X, np.array([[0.]])) # k(x,0)

# Plot covariance vector
plt.plot(X,K)

# Annotate plot
plt.xlabel("$x_i$", size=20), plt.ylabel("$\kappa$", size=20)
plt.title("$\kappa_{rbf}(x_i,0)$", size=20)

# ==== k(x_i,x_j)

plt.subplot(122) # right plot

# The kernel takes two inputs, and outputs the covariance between each respective point in the two inputs
K = k.K(X,X)

# Plot the covariance of the sample space
plt.pcolor(X.T, X, K)

# Format and annotate plot
plt.gca().invert_yaxis(), plt.gca().axis("image")
plt.xlim(int(min(X)), int(max(X))), plt.ylim(int(max(X)), int(min(X)))
plt.xlabel("$x_i$", size=20), plt.ylabel("$x_j$", size=20), plt.colorbar()
plt.title("$\kappa_{rbf}(x_i,x_j)$", size=20);

## Setting the kernel parameters

Looking at the above definition of the RBF kernel, we can see that the parameters, i.e. variance and lengthscale, control the shape of the covariance function and therefore the value of the covariance between points $x_i$ and $x_j$.

We can access the value of the kernel parameters in `GPy` and manually set them by calling `k.lengthscale` or `k.variance` for the RBF kernel. The following example demonstrates how the value of the lengthscale affects the RBF kernel. 

We only do this with our too-perfect-for-reality toy data set, because it should just demonstrate how influential these hyperparameters are.

In [ ]:
# Our toy data set and sample space: 250 samples in the interval [-4,4] 
X = np.linspace(-4.,4.,250)[:, None] # we use more samples to get a smoother plot at low lengthscales

# Create a 1-D RBF kernel with default parameters
k = GPy.kern.RBF(1)

# Set up the plotting environment
plt.figure(figsize=(18, 7))

# Set up our list of different lengthscales
ls = [0.25, 0.5, 1., 2., 4.]

# Loop over the lengthscale values
for l in ls:
    # Set the lengthscale to be l
    k.lengthscale = l
    # Calculate the new covariance function at k(x_i,0)
    C = k.K(X, np.array([[0.]]))
    # Plot the resulting covariance vector
    plt.plot(X,C)

# Annotate plot
plt.xlabel("$x_i$", size=20), plt.ylabel("$\kappa(x_i,0)$", size=20) 
plt.title("Effects of different lengthscales on the Gaussian RBF kernel", size=16)
plt.legend(labels=ls);

## GP regression model with toy data set

We will now use our Gaussian process prior, i.e. we assume all measurements come from an univariate Gaussian distribution, to learn the Gaussian process posterior with our actual measurements to form a GP regression model. 

As above, we do this with **toy data set** first and then with our real data.

The toy data set changes from the examples from above. We assume our underlying function $f(x)$ has this form:
$$
    f(x) = -\cos(2\pi x) + \frac{1}{2}\sin(6\pi x) ,
$$
but our actual measurements $y$ of this function $f(x)$ are somewhat noisy, i.e. 
$$
    y = f(x) + \epsilon, \quad \epsilon \sim \mathcal{N}(0, 0.01).
$$


Let's have a first view on this toy data set:

In [ ]:
# lambda function, call f(x) to generate data
f = lambda x: -np.cos(2*np.pi*x) + 0.5*np.sin(6*np.pi*x)

# 10 equally spaced sample locations 
X = np.linspace(0.05, 0.95, 10)[:,None]

# y = f(X) + epsilon
Y = f(X) + np.random.normal(0., 0.1, (10,1)) # note that np.random.normal takes mean and s.d. (not variance), 0.1^2 = 0.01

# Setup our figure environment
plt.figure(figsize=(18, 7))

# Plot observations
plt.plot(X, Y, "kx", mew=2)

# Annotate plot
plt.xlabel("x"), plt.ylabel("f")
plt.legend(labels=["sample points"]);

A Gaussian process regression model using a Gaussian RBF covariance function can be defined first by setting up the kernel:

In [ ]:
k = GPy.kern.RBF(1, variance=1., lengthscale=0.1, name="rbf")

And then combining it with the data to form a Gaussian process regression model, with $\mathbf{X}^*$ representing _any_ new inputs (imagine $\mathbf{f}^*$ approximates $f(\mathbf{X}^*)$):

$$
\left.\mathbf{f}^*\,\right|\,\mathbf{X}^*,\mathbf{X},\mathbf{y} \sim \mathcal{N}\left(\mathbf{m}, \mathbf{C}\right),
$$

where $
\mathbf{m} = \mathbf{K}_{*x}(\mathbf{K}_{xx} + \sigma^2\mathbf{I})^{-1}\mathbf{y}$, 

$\mathbf{C} = \mathbf{K}_{**} -  \mathbf{K}_{*x}(\mathbf{K}_{xx} + \sigma^2\mathbf{I})^{-1}\mathbf{K}_{*x}^\text{T}
$, 

and covariance matrices are defined by evaluations of the kernel functions: 

$\mathbf{K}_{xx} = k(\mathbf{X}, \mathbf{X})$; $\mathbf{K}_{*x} = k(\mathbf{X}^*, \mathbf{X})$; and $\mathbf{K}_{**} = k(\mathbf{X}^*,\mathbf{X}^*)$.

The following cell is a script which can also be done automatically with `GPy.models.GPRegression(X, Y, k)`, see a couple of cells down.

In [ ]:
# New test points to sample function from
Xnew = np.linspace(-0.05, 1.05, 100)[:, None]

# Covariance between training sample points (+ Gaussian noise)
Kxx = k.K(X,X) + 1 * np.eye(10)

# Covariance between training and test points
Kxs = k.K(Xnew, X)

# Covariance between test points
Kss = k.K(Xnew,Xnew)

# The mean of the GP fit (note that @ is matrix multiplcation: A @ B is equivalent to np.matmul(A,B))
mean = Kxs @ np.linalg.inv(Kxx) @ Y
# The covariance matrix of the GP fit
Cov = Kss - Kxs @ np.linalg.inv(Kxx) @ Kxs.T

Here we define a quick plotting utility function for our GP fits. There are a number of plotting options available in GPy, but we will use the below method, which plots the mean and $95\%$ confidence fit of a GP for a given input $\mathbf{X}^*$. Optionally, we will allow it to plot the initial training points.

In [ ]:
def plot_gp(X, m, C, training_points=None):
    """ Plotting utility to plot a GP fit with 95% confidence interval """
    # Plot 95% confidence interval 
    plt.fill_between(X[:,0],
                     m[:,0] - 1.96*np.sqrt(np.diag(C)),
                     m[:,0] + 1.96*np.sqrt(np.diag(C)),
                     alpha=0.5)
    # Plot GP mean and initial training points
    plt.plot(X, m, "-")
    plt.legend(labels=["GP fit"])
    
    plt.xlabel("years", size=20), plt.ylabel("standardised values", size=20)
    
    # Plot training points if included
    if training_points is not None:
        X_, Y_ = training_points
        plt.plot(X_, Y_, "kx", mew=2)
        plt.legend(labels=["GP fit", "sample points"])

In [ ]:
plt.figure(figsize=(14, 8))

# Plot the GP fit mean and covariance
plot_gp(Xnew, mean, Cov, training_points=(X,Y))
plt.title("Explicit (homemade) regression model fit");

We can also save effort and time by to do Gaussian process regression using `GPy`, by creating a GP regression model with sample points $(\mathbf{X}, \mathbf{Y})$ and the Gaussian RBF kernel:

In [ ]:
m = GPy.models.GPRegression(X, Y, k)
m 

We can use GPy's regression and prediction tools, which _should_ give the same result as our basic implementation:

In [ ]:
# Use GPy model to calculate the mean and covariance of the fit at Xnew
mean, Cov = m.predict_noiseless(Xnew, full_cov=True)

plt.figure(figsize=(14, 8))

# Plot the GP fit mean and covariance
plot_gp(Xnew, mean, Cov, training_points=(X,Y))
plt.title("GPy regression model fit");

## Covariance function parameter estimation

The values of kernel parameters can be estimated by maximising the likelihood of the observations. This is useful to optimise our estimate of the underlying function, without eye-balling parameters to get a good fit.

In `GPy`, the `model` objects such as `GPRegression`, have parameter optimisation functionality. We can call this as following:

In [ ]:
m.optimize()
m

We can see the selected parameters in the model table above. The regression fit with the optimised parameters can be plotted:

In [ ]:
# Get mean and covariance of optimised GP
mean, Cov = m.predict_noiseless(Xnew, full_cov=True)

# Setup the figure environment
plt.figure(figsize=(14, 8))

# Plot the GP fit mean and covariance
plot_gp(Xnew, mean, Cov, training_points=(X,Y))
plt.plot(Xnew, f(Xnew), "r:", lw=3)

### Parameter constraints

We can see in the above model that the regression model is fit to the data, as the optimiser has minimised the noise effect in the model, `Gaussian_noise.variance`$ = 1.71\times10^{-8}$. If we *know*, or can reasonably approximate, the variance of the observation noise $\epsilon$, we can fix this parameter for the optimiser, using `fix`, which in the case of the above is $0.001$. We can also limit the values that the parameters take by adding constraints. For example, the variance and lengthscale can only be positive, so calling `constrain_positive`, we can enforce this (note that this is the default constraint for GP regression anyway).

In [ ]:
# Constrain the regression parameters to be positive only
m.constrain_positive()

# Fix the Gaussian noise variance at 0.0001 
m.Gaussian_noise.variance = 0.001 # (Reset the parameter first)
m.Gaussian_noise.variance.fix()

#m.rbf.variance = 0.5
#m.rbf.variance.fix()

# Reoptimise
m.optimize()
m

We can see our constraints in the corresponding column in the above table, where "`+ve`" means we are constrained to positive values, and `fixed` means the optimiser will not try and optimise this parameter. We can see here that the variance of the noise in the model is unchanged by the optimiser. Looking at the resulting plot, we can see that we have a much more reasonable confidence in our estimate, and that the true function is hard to distinguish from samples drawn from our fit, indicating that we have reasonable approximation of the true function given noisy observations.

In [ ]:
# Get mean and covariance of optimised GP
mean, Cov = m.predict_noiseless(Xnew, full_cov=True)

# Setup our figure environment
plt.figure(figsize=(18, 14))

# The top plot shows our mean regression fit and 95% confidence intervals 
plt.subplot(211)
# Plot the GP fit mean and covariance
plot_gp(Xnew, mean, Cov, training_points=(X,Y))
plt.title("GP posterior")
plt.subplot(212)

plt.plot(Xnew, f(Xnew),"r:", lw=3)

Z  = np.random.multivariate_normal(mean[:,0], Cov, 20)
for z in Z:
    plt.plot(Xnew,z, "g-", alpha=0.2)
    
plt.xlabel("x"), plt.ylabel("f")
plt.legend(labels=["true $f(x)$", "samples from GP"]);

## GP regression model with SDG data set

Now, we'll implement everything from above with our SDG data set. We have already plotted it above, but will still do it here again to keep it in mind.

In [ ]:
# years from 2000 to 2018
X = np.array(per).reshape((-1, 1))

# standardised values for these X years
Y = np.array(dict_all_i['Algeria'][period].loc['SI_POV_DAY1']).reshape((-1, 1)) + np.random.normal(0., 0.1, (19,1))

# Setup our figure environment
plt.figure(figsize=(19, 7))

# Plot observations
plt.plot(X, Y, "kx", mew=2)

# Annotate plot
plt.xlabel("years", size=20), plt.ylabel("standardised values", size=20)
plt.xticks(np.arange(min(X), max(X)+1, 1.0))
plt.legend(labels=["measurements"]);

In [ ]:
# define kernel
k = GPy.kern.RBF(1, variance=1., lengthscale=0.1, name="rbf")

In [ ]:
# optimise kernel
# Constrain the regression parameters to be positive only
m.constrain_positive()

# Fix the Gaussian noise variance at 0.0001 
m.Gaussian_noise.variance = 0.001 # (Reset the parameter first)
m.Gaussian_noise.variance.fix()

#m.rbf.variance = 0.5
#m.rbf.variance.fix()

# optimise
m.optimize()
m

We use the short paths and make the regression with `GPy`:

In [ ]:
m = GPy.models.GPRegression(X, Y, k)
m

We use `GPy`'s prediction tool for the years and months of years which we do not have any data for:

In [ ]:
# New test points to sample function from
Xnew = np.linspace(1995, 2023, 100)[:, None]

# Use GPy model to calculate the mean and covariance of the fit at Xnew
mean, Cov = m.predict_noiseless(Xnew, full_cov=True)

plt.figure(figsize=(14, 8))

# Plot the GP fit mean and covariance
plot_gp(Xnew, mean, Cov, training_points=(X,Y))
plt.xticks(np.arange(min(Xnew), max(Xnew)+1, 1.0))
plt.title("GPy regression model fit");

In [ ]:
# issue
mean

### Training and testing

The dataset provided is separated automatically into a training and test set. We will use the training set to fit our GP and use the test data to visualise the predictive power of our GP fit.

In [ ]:
# Training data (X = input, Y = observation)
X, Y = data['X'], data['Y']

# Test data (Xtest = input, Ytest = observations)
Xtest, Ytest = data['Xtest'], data['Ytest']

# Set up our plotting environment
plt.figure(figsize=(14, 8))

# Plot the training data in blue and the test data in red
plt.plot(X, Y, "b.", Xtest, Ytest, "r.")

# Annotate plot
plt.legend(labels=["training data", "test data"])
plt.xlabel("year"), plt.ylabel("CO$_2$ (PPM)"), plt.title("Monthly mean CO$_2$ at the Mauna Loa Observatory, Hawaii");

### GP regression

First, we will try to fit a basic RBF to our data, as we have used in previous examples

In [ ]:
k = GPy.kern.RBF(1, name="rbf")

m = GPy.models.GPRegression(X, Y, k)
m.optimize()

m

In [ ]:
Xnew = np.vstack([X, Xtest])

mean, Cov = m.predict(Xnew, full_cov=True)

plt.figure(figsize=(14, 8))
plot_gp(Xnew, mean, Cov)
plt.plot(X, Y, "b.");

Let's see the plots now.

In [ ]:
mean, Cov = m.predict(Xnew, full_cov=True)

plt.figure(figsize=(18, 5))

# The left plot shows the GP fit to a subsample of our training set
plt.subplot(121)
plot_gp(Xnew, mean, Cov)
plt.plot(X, Y, "b.");
plt.gca().set_xlim([1960,1980]), plt.gca().set_ylim([310, 340])

# The right plot shows that the GP has no predictive power and reverts to 0
plt.subplot(122)
plot_gp(Xnew, mean, Cov)
plt.plot(X, Y, "b.");